## Resnet-18 Model

by Alvin Zheng and Nat Efrat-Henrici

This model is built to run on the tiny-imagenet dataset.

In [ ]:
import torchvision.models as models

#googlenet = models.googlenet(pretrained=True)
network = models.resnet18(pretrained=True)

In [ ]:
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
import torch as torch
import numpy as np
import os
    


from pathlib import Path
import pickle
import gzip
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import torchvision.transforms as transforms
import cv2

In [ ]:
# read the data
datapath = "./tiny-imagenet-200/"
trainpath = datapath + "train/"
classIDs = os.listdir(trainpath)
classIDDict = {}
for i in range(len(classIDs)):
    classIDDict[classIDs[i]] = i

classCount = len(classIDs)
sourceDim = 64
targetDim = 224
channels = 3

trainPerClass = 450   #500 original
testPerClass = 10
validPerClass = 20

imageCountTr = classCount  * trainPerClass
imageCountTs = classCount  * testPerClass
imageCountV = classCount  * validPerClass

def stackImage(image):
    return np.stack((im,)*channels, axis=-1)

def processImage(image):
    if (len(image.shape) < channels):
        image = stackImage(image)
    asfloat = image.astype('float32')
    resized = cv2.resize(asfloat, dsize=(targetDim,targetDim), interpolation=cv2.INTER_CUBIC)
    rolled = np.rollaxis(resized, 2, 0)
    tensor = torch.tensor(rolled)
    normalized = normalize(tensor)
    numpied = normalized.numpy()[:,:,:]
    return numpied

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [ ]:
#training, test, validation data
imagesTr = np.zeros((imageCountTr, channels, targetDim, targetDim), dtype="float32")
labelsTr = np.zeros(imageCountTr, dtype=int)

imagesTs = np.zeros((imageCountTs, channels, targetDim, targetDim), dtype="float32")
labelsTs = np.zeros(imageCountTs, dtype=int)

imagesV = np.zeros((imageCountV, channels, targetDim, targetDim), dtype="float32")
labelsV = np.zeros(imageCountV, dtype=int)

dataset = [(imagesTr, labelsTr), (imagesTs, labelsTs), (imagesV, labelsV)]

for i in range(len(classIDs)):
    fileNames = glob.glob(trainpath+classIDs[i]+"/images/*.JPEG")
    j = 0
    for s in dataset:
        length = int(len(s[0]) / classCount)
        for k in range(length):
            classIndex = j + k
            setIndex = i * length + k
            im = mpimg.imread(fileNames[classIndex].replace("\\", "/"))
            im = processImage(im)
            label = i
            s[0][setIndex] = im
            s[1][setIndex] = label 
        j += length
    print('Class: ' + str(i))

In [ ]:
imagesTr.shape

In [ ]:
'''
#validation data
imageCountV = 10000
imagesV = np.zeros((imageCountV, channels, targetDim, targetDim), dtype="float32")
labelsV = np.zeros(imageCountV, dtype=int)
pathV = datapath + "val/"
fileNamesV = glob.glob(pathV + "images/*.JPEG")
fpV = open(pathV + "val_annotations.txt", "r")
for i in range(len(fileNamesV)):
    newlineV = fpV.readline()
    label = newlineV.split()[1]
    filePath = fileNamesV[i].replace("\\", "/")
    imNum = int(filePath.split("/")[-1][4:-5])
    im = mpimg.imread(filePath)
    im = processImage(im)
    imagesV[i] = im
    labelsV[i] = classIDDict[label]

fpV.close()
    '''

In [ ]:
x_train, y_train = imagesTr, labelsTr
x_valid, y_valid = imagesV, labelsV
x_train, y_train, x_valid, y_valid = map(
    torch.as_tensor, (x_train, y_train, x_valid, y_valid)) 

In [ ]:
if torch.cuda.is_available(): 
    dev = "cuda:0" 
    device = torch.device(dev)
    network.to(device);

In [ ]:
torch.save(x_train, 'x_train.pt')
torch.save(y_train, 'y_train.pt')
torch.save(x_valid, 'x_valid.pt')
torch.save(y_valid, 'y_valid.pt')

In [ ]:

im = mpimg.imread('blah.jpg')
arr = np.zeros((2,3,224,224), dtype='float32')
res = cv2.resize(im, dsize=(224,224), interpolation=cv2.INTER_CUBIC)
plt.imshow(res, interpolation='nearest')
plt.show()

im2 = processImage(im)
im2.shape
type(im2)
#https://docs.fast.ai/vision.data#ImageDataLoaders.from_folder

In [ ]:
train_ds = TensorDataset(x_train, y_train)
valid_ds = TensorDataset(x_valid, y_valid)

In [ ]:
import torch

def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
    )

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b))
            
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

def preprocess1(x):
    return x.view(-1, channels, targetDim, targetDim)

def preprocess2(x, y):
    return x.view(-1, channels, targetDim, targetDim), y



In [ ]:
# define log softmax and our model output

def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

# negative loss likelihood (equivalent to cross entropy)
def nll(inp, target):
    return -inp[range(target.shape[0]), target].mean()

loss_func = F.cross_entropy
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

### Let's check our loss and accuracy after one forward pass on a batch size of 64

### Training loop

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    yb = yb.long()
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)
                
def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        model.train()
        for xb, yb in train_dl:
            xb, yb = xb.to(device), yb.to(device)
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for xb, yb in valid_dl:
                xb, yb = xb.to(device), yb.to(device)
                val_loss, nums = loss_batch(model, loss_func, xb, yb)
                epoch_val_loss +=  val_loss
            epoch_val_loss /= nums
        print(epoch, epoch_val_loss)

In [ ]:
bs = 32
lr = 2.85e-3  # learning rate
epochs = 5  # how many epochs to train for

train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

In [ ]:
train_dl = WrappedDataLoader(train_dl, preprocess2)
valid_dl = WrappedDataLoader(valid_dl, preprocess2)
'''
model = nn.Sequential(
    Lambda(preprocess1),
    nn.Conv2d(channels, 16, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
    nn.ReLU(),
    nn.Conv2d(256, imageClassCount, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.AdaptiveAvgPool2d(4),
    Lambda(lambda x: x.view(x.size(0), -1)),
)'''

opt = optim.SGD(network.parameters(), lr=lr, momentum=0.9)

In [ ]:
fit(epochs, network, loss_func, opt, train_dl, valid_dl)

In [ ]:
def adde(x, y):
    print(x+y)
def adde(x):
    print(x)

adde(2, 3)

In [ ]:
torch.save(googlenet, './googlenet.pt')

In [ ]:
from torch_lr_finder import LRFinder
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(network.parameters(), lr=1e-7, weight_decay=1e-2)
lr_finder = LRFinder(network, optimizer, criterion, device=dev)
lr_finder.range_test(train_dl, end_lr=100, num_iter=100)
lr_finder.plot() # to inspect the loss-learning rate graph
lr_finder.reset() # to reset the model and optimizer to their initial state

In [ ]:
network = torch.load('./resnet.pt')
network

In [ ]:
imagesTs

In [ ]:
def accuracy(out, yb):
    preds = torch.argmax(out, dim=1)
    return (preds == yb).float().mean()

val_acc = 0
counter = 0
with torch.no_grad():
    for xb, yb in valid_dl:
        xb, yb = xb.to(device), yb.to(device)
        out = network(xb)
        val_acc += accuracy(out, yb)
        counter += 1
print(val_acc/counter)

In [ ]:
classIDDict